# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [4]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome("/Users/Apple/pyds/chromedriver")
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height



### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [8]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [9]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 富邦人壽再調整營業時間！　10大客服提前在12點半打烊

▲富邦人壽再調整全台櫃檯服務時間。（圖／資料照）
記者陳依旻／台北報導
富邦人壽今（21）日表示，由於在疫情期間，推出視訊投保、視訊生調體檢替代方案等多項零接觸數位保險服務，也持續鼓勵客戶多加利用官網線上客戶服務和0800電話客服專線等，民眾對於以數位管道來取得保險服務的接受度與熟悉度已有所提高，臨櫃人數已降至50%以下，為持續降載公司服務人力，即日起，全台10個客服櫃台營業時間提前至12點半結束。
富邦人壽指出，客服櫃檯來客數量統計，疫情爆發前每日平均總來客數約750人，5月中旬疫情爆發後平均總來客數降至約440人，6月來客數更持續下降，每日平均總來客數約330人，由於來客數已降至50%以下，為持續降載客服人力，再加上多數民眾與保戶對於數位服務已漸趨熟悉，即日起，再次調整全台10個客服櫃檯之營業時間為「早上8點半至中午12點半」，以維護客戶及員工健康與安全。
現階段民眾若想要進行視訊投保，富邦人壽表示，只需要運用手機、平板等行動裝置，透過簡單的連線、簽名、傳送３步驟即可完成，在業務員的清楚解說與詳細指引下，富邦人壽視訊投保服務廣受50歲以下民眾青睞。
----------------------------------------------------------------------

[1] 三大亮點齊釋放！點亮越股牛市情

▲越南股市表現較其他新興市場股市相對亮眼。（圖／取自免費圖庫pixabay）
記者陳心怡／台北報導
----------------------------------------------------------------------

[2] 二手散裝船成交量年增2.95倍　慧洋賣舊船加造10艘節能船

▲慧洋海運將加造10艘第三代節能船。(圖／慧洋提供）
記者張佩芬／台北報導
散裝船運市場船舶經紀人Allied Shipbroking 最新周報顯示，今年前五個月，二手散裝船成交數量高達428艘，與去年同期的145艘相較，是2.95倍。散裝船運市場前景大好，為何賣船數量攀高？慧洋海運董事長藍俊昇表示，慧洋今年就規劃賣20